In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
from src.data_preparation import load_dataset
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

dataset_path = "../data/dataset.jsonl"
dataset = load_dataset(dataset_path)
dataset = dataset[dataset.api_errors < 10]
dataset.head()

,metadata,inputs,outputs,api_errors,conversation_id
1,"{'dataset_split': ['base'], 'ls_model_type': '...","{'messages': [{'role': 'system', 'content': 'Y...","{'message': {'role': 'assistant', 'content': '...",3,1
2,"{'dataset_split': ['base'], 'ls_model_type': '...","{'messages': [{'role': 'system', 'content': 'Y...","{'message': {'role': 'assistant', 'content': '...",0,2
3,"{'dataset_split': ['base'], 'ls_model_type': '...","{'messages': [{'role': 'system', 'content': 'Y...","{'message': {'role': 'assistant', 'content': '...",0,3
4,"{'dataset_split': ['base'], 'ls_model_type': '...","{'messages': [{'role': 'system', 'content': 'Y...","{'message': {'role': 'assistant', 'content': '...",0,4
5,"{'dataset_split': ['base'], 'ls_model_type': '...","{'messages': [{'role': 'system', 'content': 'Y...","{'message': {'role': 'assistant', 'content': '...",0,5


In [34]:
from langchain_openai import ChatOpenAI
from src.analysis.topic_clustering import process_conversations
from pathlib import Path
import pandas as pd

output_topics_df = Path("../data/topics_df.csv")

if output_topics_df.exists():
    print("Loading topics from file")
    topics_df = pd.read_csv(output_topics_df)
else:
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    small_df = dataset.to_dict(orient="records")

    output_dict = process_conversations(small_df, model)

    topics_df = pd.DataFrame(
        [
            {
                "conversation_id": conv["conversation_id"],
                "main_topic": conv["category"]["main_topic"],
                "situation": conv["category"]["situation"],
                "confidence": conv["category"]["confidence"],
                "reasoning": conv["category"]["reasoning"],
            }
            for conv in output_dict
        ]
    )
    topics_df.to_csv("../data/topics_df.csv", index=False)


topics_df

,conversation_id,main_topic,situation,confidence,reasoning
0,1,Team Dynamics,Collaboration issues,0.90,The conversation primarily revolves around the...
1,2,Performance Related,Poor quality work,0.90,The conversation primarily focused on the low ...
2,3,Project Management,Resource allocation conflicts,0.90,The conversation primarily revolves around the...
3,4,Performance Related,Poor quality work,0.90,The conversation primarily focused on the seni...
4,5,Team Dynamics,Collaboration issues,0.90,The conversation primarily revolves around a t...
5,6,Career Development,Unmet salary expectations,0.90,The conversation primarily revolves around a t...
6,7,Work Life Balance,Flexible work arrangements,0.85,The conversation revolves around a team member...
7,8,Interpersonal Conflicts,Personality clashes,0.90,The conversation revolves around a difficult d...
8,9,Career Development,Unmet salary expectations,0.80,The conversation revolves around an employee's...
9,10,Project Management,Project approach disagreements,0.90,The conversation primarily revolves around fru...


In [35]:
feedback_df = pd.read_csv("../data/quantative_df.csv")
feedback_df = feedback_df[feedback_df.api_errors < 10]
feedback_df.head()

,conversation_id,conversation_length,user_words,api_errors,feedback_Q1,feedback_Q2,feedback_Q3,overall_feedback
1,1,18,364,3,very satisfactory,extremely helpful,I like its objective approach clarifying the p...,5.0
2,2,18,138,0,very satisfactory,it helped!,I dont know what else to ask :) it was fine,5.0
3,3,28,492,0,very satisfactory,extremely helpful,I really liked the structure and how the conte...,5.0
4,4,18,138,0,very satisfactory,it helped!,I dont know what else to ask :) it was fine,5.0
5,5,40,330,0,NaN,NaN,NaN,NaN


In [36]:
merged_df = pd.merge(topics_df, feedback_df, on="conversation_id", how="left")
merged_df.head()

,conversation_id,main_topic,situation,confidence,reasoning,conversation_length,user_words,api_errors,feedback_Q1,feedback_Q2,feedback_Q3,overall_feedback
0,1,Team Dynamics,Collaboration issues,0.9,The conversation primarily revolves around the...,18,364,3,very satisfactory,extremely helpful,I like its objective approach clarifying the p...,5.0
1,2,Performance Related,Poor quality work,0.9,The conversation primarily focused on the low ...,18,138,0,very satisfactory,it helped!,I dont know what else to ask :) it was fine,5.0
2,3,Project Management,Resource allocation conflicts,0.9,The conversation primarily revolves around the...,28,492,0,very satisfactory,extremely helpful,I really liked the structure and how the conte...,5.0
3,4,Performance Related,Poor quality work,0.9,The conversation primarily focused on the seni...,18,138,0,very satisfactory,it helped!,I dont know what else to ask :) it was fine,5.0
4,5,Team Dynamics,Collaboration issues,0.9,The conversation primarily revolves around a t...,40,330,0,NaN,NaN,NaN,NaN


In [37]:
from src.analysis.topic_clustering import CATEGORIES

print("Main topics:")

for key, value in CATEGORIES.items():
    print("- ", key)

Main topics:
-  Performance Related
-  Interpersonal Conflicts
-  Career Development
-  Organizational Changes
-  Professional Conduct
-  Work Life Balance
-  Ethics Compliance
-  Project Management
-  Client Relations
-  Team Dynamics


In [43]:
# Calculate mean feedback score for each main_topic and sort
topic_feedback = merged_df.groupby("main_topic")["overall_feedback"].mean().sort_values(ascending=False)
topic_feedback = topic_feedback.fillna(0)

# Create a vertical bar chart with plotly
fig = px.bar(
    y=topic_feedback.values,
    x=topic_feedback.index,
    labels={"y": "Average Feedback Score", "x": "Topic"},
    title="Average Feedback Score by Topic",
    color=topic_feedback.values,
    color_continuous_scale="Viridis",
)

# Update layout for better readability
fig.update_layout(height=600, width=800, xaxis={"categoryorder": "total descending"}, coloraxis_showscale=False)

fig.show()

In [39]:
import plotly.express as px

# Create a figure with 2 subplots side by side
fig = make_subplots(
    rows=1,
    cols=3,
    subplot_titles=("All Topics", "Topics with Feedback", "Without Feedback"),
    specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "pie"}]],
)

# First chart - all conversations
all_counts = merged_df["main_topic"].value_counts()
fig.add_trace(go.Pie(labels=all_counts.index, values=all_counts.values, hole=0.4, name="All Topics"), row=1, col=1)

# Second chart - only conversations with feedback
filtered_counts = merged_df.dropna(subset=["overall_feedback"])["main_topic"].value_counts()
fig.add_trace(
    go.Pie(labels=filtered_counts.index, values=filtered_counts.values, hole=0.4, name="With Feedback"), row=1, col=2
)

# Third chart - only conversations without feedback
filtered_counts = merged_df[merged_df["overall_feedback"].isna()]["main_topic"].value_counts()
fig.add_trace(
    go.Pie(labels=filtered_counts.index, values=filtered_counts.values, hole=0.4, name="Without Feedback"), row=1, col=3
)

# Update layout
fig.update_layout(title_text="Topic Distribution: All Conversations vs With Feedback", height=500, width=1000)

fig.show()

From the llm topic analysis we could make following observations:
- performance-related and Attendance-related topics got 3.0 average feedback score - 

In [31]:
merged_df[merged_df["overall_feedback"].isna() & merged_df["main_topic"].str.contains("Career Development")][
    "conversation_id"
].tolist()

[6, 9, 13, 18]

In [68]:
merged_df[merged_df["main_topic"].str.contains("Performance")]

,conversation_id,main_topic,situation,confidence,reasoning,conversation_length,number_of_user_words,api_errors,feedback_Q1,feedback_Q2,feedback_Q3,conversation_length_without_feedback,overall_feedback
1,2,Performance Related,Poor quality work,0.9,The conversation primarily focused on the seni...,26,155,0,it helped!,I dont know what else to ask :) it was fine,good night!,20,3.0
3,4,Performance Related,Poor quality work,0.9,The conversation primarily revolved around the...,24,153,0,it helped!,I dont know what else to ask :) it was fine,NaN,20,3.0
10,11,Performance Related,Productivity concerns,0.9,The conversation primarily focused on Andrea's...,22,170,0,NaN,NaN,NaN,22,NaN
11,12,Performance Related,Productivity concerns,0.9,The conversation revolves around a client esca...,14,223,0,NaN,NaN,NaN,14,NaN
